You have a number of envelopes with widths and heights given as a pair of integers `(w, h)`. One envelope can fit into another if and only if both the width and height of one envelope is greater than the width and height of the other envelope.

What is the maximum number of envelopes can you Russian doll? (put one inside other)

Note:
Rotation is not allowed.

So right off the bat I think I should sort the list. Though perhaps it wouldn't matter. In theory, if we had a table of which envelopes fit inside other envelopes, then we could just recursively iterate and find the answer.

After beginning to think about the problem, I decided the direction to proceed is to let DP(i) be the number of maximum number of envelopes involving the ith envelope. Then DP(j) is the maximum over all DP(i) where i < j and `envelopes[j]` contains `envelopes[i]` and 1 (the case where the jth envelope doesn't contain any earlier envelopes).

In [29]:
class Solution:
    def maxEnvelopes(self, envelopes) -> int:
        n = len(envelopes)
        envelopes.sort() # O(n*log(n)), n is the length of envelopes
        
        D = []
        for i, (a, b) in enumerate(envelopes):
            max_envelope = 1
            for j in range(i):
                c, d = envelopes[j]
                if c < a and d < b:
                    max_envelope = max(max_envelope, D[j]+1)
            D.append(max_envelope)
        # O(n**2)
        
        # We handle the case when n == 0
        D.append(0)
        
        return max(D)

9168 ms (5.08%) and 15.1 MB (20.00%)  
Runtime Complexity: O(n\*\*2) where `n` is the number of envelopes.  
Extra Space Complexity: O(n)

One of the main problems of the above is that it makes some unnecessary envelope containment checks. For example if `envelopes[j]` contains `envelopes[i]` and `envelopes[k]` contains `envelopes[j]` then we don't need to check the relationship between `envelopes[i]` and `envelopes[k]`.

What we can do instead is create a directed graph and only check containment of a node's children if the parent node is not contained.

In [81]:
class Solution:
    def maxEnvelopes(self, envelopes) -> int:
        if not envelopes:
            return 0
        n = len(envelopes)
        envelopes.sort()
        
        parents = []
        node_children = {}
        
        def is_one_inside_two(envelope1, envelope2, visited):
            if envelope1[0] < envelope2[0] and envelope1[1] < envelope2[1]:
                node_children.setdefault(envelope2, {}).setdefault("children", []).append(envelope1)
                return (node_children[envelope1]["max"], True)
            else:
                res = 0
                for child in node_children[envelope1].get("children", []):
                    if child not in visited:
                        visited.add(child)
                        res = max(res, is_one_inside_two(child, envelope2, visited)[0])
                return (res, False)
                    
        for i, (a, b) in enumerate(envelopes):
            max_envelope = 0
            new_parents = []
            for parent in list(parents):
                res, is_contained = is_one_inside_two(parent, (a, b), set())
                max_envelope = max(res, max_envelope)
                if not is_contained:
                    new_parents.append(parent)
            new_parents.append((a, b))
            parents = new_parents
            node_children.setdefault((a, b), {}).setdefault("max", max_envelope+1)
        
        return max([node_children[parent]["max"] for parent in parents])

Time Limit Exceeded
Worst-case: O(n\*\*2)  
Best-case: O(n\*log(n))

So I did some thinking and realzied that if two envelopes have the same width, then they belong to the same bucket. Then it is a matter of being greedy with the height.

In [85]:
class SolutionTester(Solution):
    def test(self):
        testcase1 = ([[5,4],[6,4],[6,7],[2,3]], 3)     # Example on LeetCode
        
        testcase2 = ([], 0)
        testcase3 = ([[1,2]], 1)
        testcase4 = ([[1,2],[2,1]], 1)
        testcase5 = ([[1,2],[1,3]], 1)
        testcase6 = ([[2,3],[3,3]], 1)
        testcase7 = ([[1,2],[2,3],[2,4],[3,2],[3,3],[3,4]], 3)
        testcase8 = ([[1,2],[2,3],[2,4],[3,4],[4,5],[5,6]], 5)
        
        # Failed test case 75/85 on LeetCode;
        # I get the right answer but "Time Limit Exceeded" 
        testcase9 = ([[733,302],[127,525],[265,484],[104,428],[682,351],[627,369],[394,657],[101,778],[142,113],[73,718],[248,637],[613,211],[654,759],[243,267],[45,28],[108,289],[492,267],[924,914],[445,250],[847,180],[928,648],[230,806],[22,527],[963,744],[674,45],[46,996],[883,623],[376,945],[762,648],[357,423],[892,280],[205,421],[721,648],[66,836],[180,800],[79,452],[243,115],[752,2],[766,984],[907,706],[329,173],[843,289],[505,199],[545,930],[56,254],[888,189],[320,927],[177,412],[887,195],[415,189],[435,764],[814,202],[1000,77],[568,961],[472,645],[447,273],[591,844],[313,663],[297,279],[690,108],[29,1],[808,103],[441,519],[655,252],[614,105],[178,486],[136,913],[102,567],[823,358],[177,750],[366,147],[755,408],[912,540],[198,302],[293,761],[508,268],[707,250],[887,692],[312,134],[634,663],[351,5],[66,849],[731,423],[695,615],[523,823],[177,441],[94,513],[861,389],[364,313],[355,868],[593,923],[783,894],[485,446],[150,196],[806,128],[5,229],[454,340],[889,527],[87,69],[720,374],[266,889],[179,944],[330,866],[882,580],[255,589],[139,138],[632,439],[90,152],[556,178],[883,203],[715,104],[902,439],[559,294],[445,193],[509,328],[546,787],[717,53],[184,366],[615,630],[186,783],[613,937],[354,477],[234,366],[200,222],[624,337],[533,561],[467,216],[923,405],[959,740],[180,858],[128,939],[621,355],[754,197],[571,214],[652,222],[774,324],[469,216],[604,61],[679,946],[896,439],[722,560],[361,498],[868,966],[961,49],[720,936],[869,852],[18,226],[728,601],[772,277],[186,8],[592,733],[410,838],[352,458],[398,332],[402,344],[37,228],[3,645],[314,52],[235,136],[149,451],[413,875],[220,634],[889,626],[192,252],[814,74],[151,84],[16,293],[133,424],[669,766],[133,790],[724,756],[248,331],[883,463],[814,387],[279,401],[654,399],[988,110],[36,779],[129,482],[614,752],[848,143],[932,741],[160,633],[85,80],[637,285],[802,634],[696,646],[850,892],[748,808],[662,971],[78,448],[325,246],[968,863],[717,393],[359,881],[498,587],[205,629],[106,902],[297,172],[934,614],[895,58],[903,111],[248,894],[828,289],[496,957],[936,609],[660,541],[731,648],[54,804],[982,16],[36,827],[110,84],[595,5],[235,297],[210,249],[483,983],[45,382],[738,840],[407,969],[599,622],[152,59],[731,77],[171,490],[458,160],[463,526],[580,957],[897,303],[280,689],[264,732],[486,391],[184,31],[391,208],[543,139],[770,252],[460,860],[63,827],[673,26],[844,1],[401,469],[217,116],[952,848],[104,604],[967,871],[33,751],[318,195],[448,297],[718,966],[638,328],[827,837],[104,412],[836,658],[371,655],[760,120],[212,893],[42,711],[44,459],[148,98],[789,577],[725,76],[338,281],[40,19],[520,143],[680,622],[616,603],[531,232],[532,284],[269,773],[305,200],[650,95],[479,90],[598,614],[137,13],[834,380],[14,390],[518,564],[747,467],[278,837],[953,102],[344,666],[458,930],[990,984],[940,321],[476,156],[958,393],[543,486],[743,705],[621,868],[415,559],[45,536],[794,669],[521,812],[659,57],[893,123],[886,129],[545,566],[802,220],[142,280],[521,474],[228,139],[474,322],[230,882],[227,653],[119,706],[83,712],[919,407],[777,4],[142,590],[564,470],[503,426],[754,485],[997,258],[166,154],[688,63],[716,357],[509,886],[159,459],[383,204],[354,517],[560,224],[245,161],[62,21],[97,105],[214,951],[990,484],[109,937],[37,916],[822,262],[164,990],[512,741],[661,70],[163,34],[439,672],[960,95],[741,63],[791,941],[631,398],[892,739],[997,818],[385,255],[879,639],[956,36],[987,349],[241,396],[986,817],[277,496],[412,128],[96,882],[938,425],[877,259],[278,437],[832,891],[96,57],[832,443],[482,974],[451,433],[690,443],[849,235],[697,937],[639,377],[545,477],[559,745],[387,946],[232,588],[599,823],[591,328],[63,810],[829,343],[780,269],[567,140],[36,779],[524,11],[304,147],[912,530],[147,955],[839,220],[297,853],[517,715],[117,338],[860,625],[850,923],[34,123],[660,534],[575,365],[355,174],[220,619],[267,320],[63,20],[124,477],[30,384],[407,92],[708,851],[515,142],[557,60],[369,618],[900,772],[172,641],[979,576],[165,541],[338,281],[456,502],[59,362],[404,771],[691,183],[981,919],[259,319],[62,14],[791,648],[459,371],[600,475],[109,630],[652,831],[368,843],[898,585],[22,273],[124,196],[30,375],[274,544],[288,380],[744,284],[962,260],[164,719],[841,937],[411,143],[789,156],[683,1],[176,380],[559,578],[896,261],[274,909],[347,365],[892,9],[560,856],[25,876],[794,771],[642,223],[254,336],[309,509],[12,824],[326,606],[742,806],[674,540],[765,719],[678,18],[803,932],[300,448],[660,330],[660,165],[298,998],[915,643],[387,201],[521,137],[952,784],[438,382],[488,453],[35,482],[715,942],[233,512],[780,669],[386,782],[794,561],[77,776],[749,146],[61,36],[619,800],[996,606],[842,163]], 40)
        
        testcases = [testcase1, testcase2, testcase3,
                     testcase4, testcase5, testcase6, 
                     testcase7, testcase8, testcase9]
        
        for test_input, expected_output in testcases:
            print("Input:", test_input)
            sol_output = self.maxEnvelopes(test_input)
            print("Output:", sol_output)
            print("Expected Output:", expected_output)
            assert sol_output == expected_output, "Testcase failed!"
            print()

In [86]:
s = SolutionTester()

In [87]:
s.test()

Input: [[5, 4], [6, 4], [6, 7], [2, 3]]
Output: 3
Expected Output: 3

Input: []
Output: 0
Expected Output: 0

Input: [[1, 2]]
Output: 1
Expected Output: 1

Input: [[1, 2], [2, 1]]
Output: 1
Expected Output: 1

Input: [[1, 2], [1, 3]]
Output: 1
Expected Output: 1

Input: [[2, 3], [3, 3]]
Output: 1
Expected Output: 1

Input: [[1, 2], [2, 3], [2, 4], [3, 2], [3, 3], [3, 4]]
Output: 3
Expected Output: 3

Input: [[1, 2], [2, 3], [2, 4], [3, 4], [4, 5], [5, 6]]
Output: 5
Expected Output: 5

Input: [[733, 302], [127, 525], [265, 484], [104, 428], [682, 351], [627, 369], [394, 657], [101, 778], [142, 113], [73, 718], [248, 637], [613, 211], [654, 759], [243, 267], [45, 28], [108, 289], [492, 267], [924, 914], [445, 250], [847, 180], [928, 648], [230, 806], [22, 527], [963, 744], [674, 45], [46, 996], [883, 623], [376, 945], [762, 648], [357, 423], [892, 280], [205, 421], [721, 648], [66, 836], [180, 800], [79, 452], [243, 115], [752, 2], [766, 984], [907, 706], [329, 173], [843, 289], [505, 19